# Analyze Product Sentiment

In [1]:
import turicreate

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


# Read product review data

In [2]:
products = turicreate.SFrame('/home/scott/Documents/ML_WASH/week3/amazon_baby.sframe/amazon_baby.sframe/m_bfaa91c17752f745.frame_idx')

# Explore data

In [3]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

/usr/local/lib/python3.6/dist-packages/turicreate/aggregate.py:92: UserWarning: Passing parameter(s) to COUNT is deprecated. This functionality will be removed in the next major release. Any passed parameters are ignored.
  + 'the next major release. Any passed parameters are ignored.')


name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

## Build word count vectors

In [4]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [14]:
df = products.to_dataframe()

In [25]:
for word in selected_words:
    print(word)
    print(sum(products[word]))

awesome
3892
great
55791
fantastic
1664
amazing
2628
love
41994
horrible
1110
bad
4183
terrible
1146
awful
687
wow
425
hate
1107


In [5]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

for col in selected_words:
    products[col] = 0
    
products = products[products['rating']!= 3]
products['sentiment'] = products['rating'] >= 4

In [6]:
def extract_words(row):
    return row['word_count'].get(word, 0)

for word in selected_words:
    products[word] = products.apply(extract_words)

    
train_data,test_data = products.random_split(.8, seed=0)
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=selected_words, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 1.564959     | 0.847708          | 0.846385            |

| 5         | 6        | 1.680348     | 0.847708          | 0.846385            |

| 6         | 7        | 1.806319     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [10]:
selected_words_model.coefficients.sort('value')

name,index,class,value,stderr
horrible,None,1,-2.251335236759102,0.08020249388788425
terrible,None,1,-2.2236614360851346,0.07731736203785744
awful,None,1,-2.0529082040313598,0.10099735435259247
hate,None,1,-1.3484407222463202,0.07715698604297323
bad,None,1,-0.9914778800650664,0.03848428664699063
wow,None,1,-0.00953823606768843,0.16046411224711649
great,None,1,0.8630655001196439,0.018955052444376685
fantastic,None,1,0.88580475688142,0.11167591293399651
amazing,None,1,1.100093311366018,0.0995477626046598
awesome,None,1,1.133534666034134,0.08399643983187562


In [11]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.39622654670876317,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 53

# Define what is positive and negative sentiment

In [13]:
products['rating'].show()

Materializing SArray

In [9]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [16]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1,'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1, 'of': 1,'the': 1, 'than': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'approach': 2, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1, 'now': 1,'its': 1, 'stick': 1, ...",1


In [17]:
products['sentiment'].show()

Materializing SArray

# Train our sentiment classifier

In [18]:
train_data,test_data = products.random_split(.8,seed=0)

In [26]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 2.679732     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 3.169083     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 3.852928     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 5.888443     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [28]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9342357833151299,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.3304787187232111,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0

In [31]:
test_data = test_data[test_data['rating']!= 3]
test_data['sentiment'] = test_data['rating'] >= 4

len(test_data['sentiment'] == 1) / len(test_data)

1.0

In [35]:
test_data['sentiment'].sum() / len(test_data)

0.8400192169108815

# Apply the sentiment classifier to better understand the Giraffe reviews

In [39]:
products['predicted_sentiment'] = selected_words_model.predict(products, output_type = 'probability')

In [40]:
products[products['name'] == 'Baby Trend Diaper Champ']

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'convenient': 1, 'more':1, 'trash': 1, 'regul ...",0,0,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'system': 1, 'try': 1,'re': 1, 'still': 1, ...",0,0,0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'nose': 1, 'for': 2,'investment': 1, 'wise': ...",0,0,0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'out': 1, 'pull': 1,'open': 1, 'pail': 1, ...",0,0,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'winter': 1, 'outside':1, 'day': 1, 'or': 1, ...",0,0,0
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'mom': 1, 'my': 1,'empty': 2, 'poop': 1, ...",0,0,0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'yuck': 1, 'clean': 1,'all': 1, 'trash': 3, ...",0,0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1,'suggestions': 1, ...",0,0,0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'best': 1, 'that': 1,'handle': 1, 'in': 1, ...",0,0,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'no': 1, 'regular': 1,'part': 1, 'important': ...",0,0,0


In [22]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether']

In [23]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'purchase': 1,'teething': 1, 'cranky': ...",1
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'a': 1, 'is': 1, 'when':1, 'him': 1, 'help': 1, ...",1
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'made': 1, 'of': 1,'packaging': 1, 'no': 1, ...",0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'another': 1, 'out': 1,'run': 1, 'lost': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'disappointed': 1,'will': 1, 'take': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'late': 1, 'perfect': 1,'pack': 1, 'only': 1, ...",1
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'delight': 1, 'in': 1,'other': 1, 'sophie': 1, ...",1
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'inside': 1, 'water': 1,'don': 1, 'up': 1, ...",1
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'its': 1, 'fine': 1,'is': 1, 'which': 1, ...",1
Vulli Sophie the GiraffeTeether ...,This product is without adoubt the best on the ...,5.0,"{'belongs': 1, 'dog': 1,'if': 1, 'beware': 1, ...",1


# Sort the Giraffe reviews according to predicted sentiment

In [24]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [25]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'will': 1, '15months':1, 'would': 2, 'range': ...",1
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'around': 1, 'explore':1, 'they': 1, ...",1
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'11': 1, 'prisrob': 1,'12': 1, 'who': 1, ...",1
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'out': 1, 'would': 1,'ve': 1, 'enough': 1, ...",1
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5.0,"{'whether': 1, 'neutral':1, 'gender': 1, 'glad': ...",1
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5.0,"{'penny': 1, 'little': 1,'perfect': 1, 'no': 1, ...",1
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4.0,"{'worth': 1, 'works': 1,'teether': 1, 'want': 1, ...",1
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5.0,"{'question': 1, 'must':1, 'overall': 1, 'goes': ...",1
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'makers': 1, 'or': 1,'take': 1, 'can': 1, ...",1
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4.0,"{'dye': 1, 'of': 1,'cause': 1, 'fade': 1, ...",1


In [26]:
giraffe_reviews.tail()

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,My 7 1/2 month oldgrandbaby was sittng up ...,1.0,"{'sopie': 1, 'with': 1,'be': 1, 'should': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I was so looking forwardto getting this for my ...,1.0,"{'amazon': 1, 'back': 1,'by': 1, 'more': 1, ...",0
Vulli Sophie the GiraffeTeether ...,Received the product andsmells like cheap rub ...,1.0,"{'was': 1, 'as': 1,'still': 1, 'but': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I got one of these as ashowe gift that my baby ...,1.0,"{'typical': 1, 'paint':1, 'came': 1, 'mine': 1, ...",0
Vulli Sophie the GiraffeTeether ...,When we first got it Ithought did my ...,2.0,"{'mom': 1, 'teethers': 1,'btw': 1, 'other': 1, ...",0
Vulli Sophie the GiraffeTeether ...,When I received this thepaint was peeling off in ...,1.0,"{'risk': 1, 'but': 1,'help': 1, 'terrible' ...",0
Vulli Sophie the GiraffeTeether ...,"""Sophie the Giraffe"" hastested positive for ...",1.0,"{'ingest': 1, 'they': 1,'naturally': 1, 'babi ...",0
Vulli Sophie the GiraffeTeether ...,"My son (now 2.5) LOVEDhis Sophie, and I bought ...",1.0,"{'exaggerated': 1, 'are':1, 'a': 3, 't': 2, ...",0
Vulli Sophie the GiraffeTeether ...,I wanted to love thisproduct and was excited ...,1.0,"{'risk': 1, 'ready': 1,'bummed': 1, 'just': 1, ...",0
Vulli Sophie the GiraffeTeether ...,This children's toy isnostalgic and very cute. ...,1.0,"{'amazon': 1, 'from': 1,'one': 1, 'by': 1, 'be': ...",0


## Show the most positive reviews

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

# Most negative reivews

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [ ]:
giraffe_reviews[-2]['review']